## Overview

For this submission, the Tree-based Optimisation Pipeline (TPOT) package was usedto automate the creation of a regressor for the steel dataset.

This process was carried out as follows:
1. In a local project repository, the tpot_engine Python script was run to perform the following steps:
    - Import the steel data
    - Transform the composition attribute into an array of element percentages, adding zeroes where no data was recordedfor a given element
    - Perform a train test split
    - Use the train and test data to optimise a TPOTRegressor
    - Export the best resulting TPOT pipeline as a pickled object
2. In the Matbench repository, the cell beneath was then run to 
    - Load the pickled model
    - For each fold:
        - Load the foldwise data, and process it
        - Perform regression using the loaded model
    - Write the results to results.json
    - Load the summary statistics for the results obtained.

In [1]:
import sys
import pickle
import joblib
import re
sys.path.insert(0,'../..')
from matbench.bench import MatbenchBenchmark
import pandas as pd
import numpy as np
import json
from utils import LoadExisting
 
regressor = pickle.load(open('tpot_best_pipeline.pkl', 'rb'))

mb = MatbenchBenchmark(autoload=False, subset=['matbench_steels'])

for task in mb.tasks:
    print(mb.tasks)
    print(task)
    task.load()

    for fold in task.folds:
        X_train, y_train = task.get_train_and_val_data(fold)
        X_test = task.get_test_data(fold, include_target=False)
        loader = LoadExisting()
        X_test = loader.cleaning(X_test)
        X_train = loader.cleaning(X_train)
        regressor.fit(np.array(X_train, dtype=np.float64), y_train)
        predictions = regressor.predict(np.array(X_test, dtype=np.float64))
        task.record(fold, predictions)
#task.validate()
# Save your results
mb.to_file("results.json.gz")

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
C:\Users\d.molloy\Anaconda3\envs\automl_extension_tpot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: issubclass() arg 1 must be a class

In [ ]:
import json
import pandas as pd
with open('results.json.gz') as f:
    jsonstr = json.load(f)

df = pd.io.json.json_normalize(jsonstr)
df.filter(regex='scores').filter(regex='mae')